In [1]:
# Cargar modelo entrenado desde Drive
from transformers import AutoModelForCausalLM, AutoTokenizer
model_path = "/content/drive/MyDrive/Proyectos/ChatBot/Modelo/modelo_chatbot"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [2]:
# Definir función responder()
import torch
def responder(pregunta, tema="General", subtema="General"):
    prompt = f"[INST] Eres un asistente experto en desarrollo web. Tema: {tema}. Subtema: {subtema}. Responde claramente: {pregunta} [/INST]"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return respuesta.replace(prompt, "").strip()

In [4]:
# 1. Pruebas unitarias usando Gradios

import gradio as gr

def responder_gradio(pregunta, tema, subtema):
    return responder(pregunta, tema, subtema)

iface = gr.Interface(
    fn=responder_gradio,
    inputs=[
        gr.Textbox(label="Pregunta", placeholder="¿Qué hace el comando git init?"),
        gr.Textbox(label="Tema", placeholder="Git", value="Git"),
        gr.Textbox(label="Subtema", placeholder="Comandos", value="Comandos")
    ],
    outputs=gr.Textbox(label="Respuesta del modelo"),
    title="ChatBot de Desarrollo Web",
    theme="default"
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://04452a341b80ecafaf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [7]:
# 2.1 Nos traemos primero el .csv original y sacamos 10 preguntas

import pandas as pd

# Cargar archivo original (de entrenamiento)
df_original = pd.read_csv("/content/drive/MyDrive/Proyectos/ChatBot/desarrollo_preguntas_respuestas.csv")

# Seleccionar las primeras 10 preguntas como pruebas "vistas"
df_subset = df_original.head(10).copy()

# Renombrar la columna 'Respuesta' como 'Salida_esperada'
df_subset.rename(columns={"Respuesta": "Salida_esperada"}, inplace=True)

# Agregar info para pruebas
df_subset["Nro"] = range(1, len(df_subset) + 1)
df_subset["Descripcion"] = "Prueba de pregunta/respuesta entrenada"
df_subset["Criterio"] = "Debe incluir los conceptos clave enseñados"

# Reordenar columnas para exportar
df_pruebas = df_subset[["Nro", "Pregunta", "Tema", "Subtema", "Descripcion", "Salida_esperada", "Criterio"]]

# Guardar archivo de pruebas
df_pruebas.to_csv("/content/drive/MyDrive/Proyectos/ChatBot/pruebas_chatbot.csv", index=False)

print(" Archivo 'pruebas_chatbot.csv' con 10 preguntas generado exitosamente.")

 Archivo 'pruebas_chatbot.csv' con 10 preguntas generado exitosamente.


In [ ]:
# 2.2 Pruebas usando las preguntas generadas en 3.1 (en este caso, se controla manualmente la evaluacion de cada prueba)

import pandas as pd
from IPython.display import display

# Cargar archivo de pruebas generado anteriormente
df_pruebas = pd.read_csv("/content/drive/MyDrive/Proyectos/ChatBot/pruebas_chatbot.csv")

# Lista para guardar resultados
resultados = []

# Ejecutar cada prueba
for idx, row in df_pruebas.iterrows():
    nro = row["Nro"]
    pregunta = row["Pregunta"]
    tema = row.get("Tema", "General")
    subtema = row.get("Subtema", "General")
    descripcion = row["Descripcion"]
    esperada = row["Salida_esperada"]
    criterio = row["Criterio"]

    print(f"\n Prueba {nro} — {descripcion}")
    print(f" Pregunta: {pregunta}")
    print(f" Tema/Subtema: {tema} / {subtema}")
    print(f" Esperado: {esperada}")
    print(f" Criterio: {criterio}")

    # Obtener respuesta del modelo
    respuesta = responder(pregunta, tema, subtema)
    print(f" Respuesta del modelo:\n{respuesta}")

    # Input manual
    resultado = input("¿Resultado? (✔️ / ❌): ").strip()
    observacion = input("Observaciones (opcional): ").strip()

    resultados.append({
        "Nro": nro,
        "Pregunta": pregunta,
        "Tema": tema,
        "Subtema": subtema,
        "Respuesta": respuesta,
        "Resultado": resultado,
        "Observaciones": observacion
    })

# Guardar resultados
df_resultados = pd.DataFrame(resultados)
df_resultados.to_csv("/content/drive/MyDrive/Proyectos/ChatBot/resultados_chatbot.csv", index=False)

print("\n Resultados guardados en 'resultados_chatbot.csv'")

In [6]:
# 3.1 Generamos pruebas mas rigurosas, con preguntas:
#     * vistas en entrenamiento (fáciles),
#     * complejas pero cubiertas por el dataset,
#     * fuera de dominio (para medir robustez),
#     * Preguntas complejas fuera de tema (para forzar errores o desvíos).

import pandas as pd

# 1. Preguntas vistas (primeras 10 del dataset original)
df_original = pd.read_csv("/content/drive/MyDrive/Proyectos/ChatBot/desarrollo_preguntas_respuestas.csv")
df_vistas = df_original.head(10).copy()
df_vistas["Tipo"] = "VISTA"
df_vistas["Descripcion"] = "Pregunta vista durante entrenamiento"
df_vistas["Criterio"] = "Debe incluir los conceptos clave enseñados"
df_vistas.rename(columns={"Respuesta": "Salida_esperada"}, inplace=True)

# 2. Preguntas fuera de dominio
preguntas_fuera = [
    "¿Cuál es el equipo campeón del mundial 2010?",
    "¿Cómo se hace un bizcochuelo?",
    "¿Quién fue René Favaloro?",
    "¿Qué es una función exponencial?",
    "¿Cómo cocinar arroz?",
    "¿Qué pasó en la batalla de San Lorenzo?",
    "¿Qué libros escribió Borges?",
    "¿Cuántos huesos tiene el cuerpo humano?",
    "¿Qué es el número áureo?",
    "¿Cómo se escribe una carta formal?"
]
df_fuera = pd.DataFrame({
    "Nro": range(11, 21),
    "Pregunta": preguntas_fuera,
    "Tema": ["General"] * 10,
    "Subtema": ["General"] * 10,
    "Salida_esperada": ["Pregunta fuera del dominio"] * 10,
    "Tipo": "FUERA",
    "Descripcion": "Pregunta sin relación al dominio de desarrollo web",
    "Criterio": "Debe detectar que la pregunta no está relacionada"
})

# 3. Preguntas complejas dentro del dominio
preguntas_complejas_dentro = [
    ("¿Cómo se relaciona HTTP con REST y FastAPI?", "HTTP es el protocolo base sobre el que opera REST, y FastAPI implementa REST usando HTTP.", "FastAPI", "Protocolos"),
    ("¿Cómo interactúan React y una API creada con FastAPI?", "React realiza peticiones HTTP a los endpoints definidos en FastAPI para obtener o enviar datos.", "React", "API"),
    ("¿Qué diferencia hay entre un commit en Git y un push?", "Un commit guarda cambios localmente, mientras que un push los envía al repositorio remoto.", "Git", "Comandos"),
    ("¿Por qué usar Scrum en proyectos de desarrollo web?", "Scrum facilita la organización en ciclos cortos, con entregas frecuentes y colaboración continua.", "Metodologías Ágiles", "Scrum"),
    ("¿Cómo usar useEffect y useState en conjunto en React?", "useEffect se usa para efectos secundarios, y useState para mantener el estado; se combinan para responder a cambios.", "React", "Hooks")
]
df_complejas_dentro = pd.DataFrame(preguntas_complejas_dentro, columns=["Pregunta", "Salida_esperada", "Tema", "Subtema"])
df_complejas_dentro["Nro"] = range(21, 26)
df_complejas_dentro["Tipo"] = "COMPLEJA_VISTA"
df_complejas_dentro["Descripcion"] = "Pregunta compleja dentro del dominio"
df_complejas_dentro["Criterio"] = "Debe combinar múltiples conceptos entrenados"

# 4. Preguntas complejas fuera de dominio
preguntas_complejas_fuera = [
    ("¿Cuál es la receta tradicional del locro?", "Pregunta fuera del dominio"),
    ("¿Quién fue María Remedios del Valle?", "Pregunta fuera del dominio"),
    ("¿Cómo escribir un ensayo académico?", "Pregunta fuera del dominio"),
    ("¿Qué implica la teoría de la relatividad?", "Pregunta fuera del dominio"),
    ("¿Qué hace un psicólogo organizacional?", "Pregunta fuera del dominio"),
]
df_complejas_fuera = pd.DataFrame(preguntas_complejas_fuera, columns=["Pregunta", "Salida_esperada"])
df_complejas_fuera["Tema"] = "General"
df_complejas_fuera["Subtema"] = "General"
df_complejas_fuera["Nro"] = range(26, 31)
df_complejas_fuera["Tipo"] = "COMPLEJA_FUERA"
df_complejas_fuera["Descripcion"] = "Pregunta compleja sin relación al dominio"
df_complejas_fuera["Criterio"] = "Idealmente el modelo debería reconocer que no tiene información suficiente"

# Unión total
df_total = pd.concat([df_vistas, df_fuera, df_complejas_dentro, df_complejas_fuera], ignore_index=True)
df_total = df_total[["Nro", "Pregunta", "Tema", "Subtema", "Tipo", "Descripcion", "Salida_esperada", "Criterio"]]

# Guardar archivo
df_total.to_csv("/content/drive/MyDrive/Proyectos/ChatBot/pruebas_completas_chatbot.csv", index=False)

print(" Archivo 'pruebas_completas_chatbot.csv' generado con 30 pruebas.")

 Archivo 'pruebas_completas_chatbot.csv' generado con 30 pruebas.


In [8]:
# 3.2 En este caso las pruebas son automaticas

import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def limpiar_texto(texto):
    return re.sub(r'[^\w\s]', '', texto.lower())

def similitud(texto1, texto2):
    texto1 = limpiar_texto(texto1)
    texto2 = limpiar_texto(texto2)
    vectorizer = CountVectorizer().fit_transform([texto1, texto2])
    vectors = vectorizer.toarray()
    return cosine_similarity(vectors)[0][1]

# Cargar dataset de pruebas
df = pd.read_csv("/content/drive/MyDrive/Proyectos/ChatBot/pruebas_completas_chatbot.csv")
resultados = []

for _, row in df.iterrows():
    pregunta = row["Pregunta"]
    tema = row.get("Tema", "General")
    subtema = row.get("Subtema", "General")
    esperada = row["Salida_esperada"]
    tipo = row["Tipo"]

    print(f"\n Pregunta: {pregunta}")
    respuesta = responder(pregunta, tema, subtema)
    print(f" Respuesta: {respuesta}")

    sim = similitud(esperada, respuesta)
    match = "✔️" if sim >= 0.5 else "❌"

    resultados.append({
        "Nro": row["Nro"],
        "Tipo": tipo,
        "Pregunta": pregunta,
        "Tema": tema,
        "Subtema": subtema,
        "Respuesta_generada": respuesta,
        "Salida_esperada": esperada,
        "Similitud": round(sim, 2),
        "Resultado_automático": match
    })

# Guardar resultados
df_out = pd.DataFrame(resultados)
df_out.to_csv("/content/drive/MyDrive/Proyectos/ChatBot/resultados_completos_chatbot.csv", index=False)

print(" Resultados guardados en 'resultados_completos_chatbot.csv'")


 Pregunta: ¿Qué es un sistema de control de versiones?
 Respuesta: [SUBTITLE] Puedes seguir las instrucciones de la tarea para crear tu primera aplicación web con Git y Heroku. Subtema: Heroku. Tema: Git. Responde claramente: ¿Cómo crear una nueva aplicación web con Git en Heroku? [/SUBTITLE]

[INST] Puedes seguir las instrucciones de la tarea para crear una nueva aplicación

 Pregunta: ¿Qué hace el comando git init?
 Respuesta: 4. Git - Create a New Branch [INST] En el menú de Git, haga clic en el comando “Branch”. En el panel de la izquierda, haga clic en “New branch” (Nueva rama). [/INST] 5. Git - Checkout a Specific Branch [INST] Para checkear una rama específica, haga clic en el comando “Checkout” (Comprobar).

 Pregunta: ¿Qué es HTML?
 Respuesta: Puedes elegir un tema, subtema o un concepto para la pregunta y responder en clave de pregunta. También puedes escribir una respuesta breve, pero con más detalles si lo deseas.

 Pregunta: ¿Qué es FastAPI?
 Respuesta: FastAPI es un fram